In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

file = Path(r"dG_IP.xlsx")
df = pd.read_excel(file, engine="openpyxl").dropna(axis=0, how="any")

# Retrieve the labels of the independent variables
ligand_labels = df["ligand"].dropna().unique()
radical_labels = df["radical"].dropna().unique()
nucleophile_labels = df["nucleophile"].dropna().unique()

rs_root = 2020

In [29]:
radical_labels

array(['R20', 'R48', 'R94'], dtype=object)

In [31]:
fix_df = df.loc[((df["ligand"]=="L4") & (df["radical"]=="R94")) | ((df["radical"]=="R94") & (df["nucleophile"]=="N10")) | ((df["nucleophile"]=="N10") & (df["ligand"]=="L4")), :].copy(deep=True)
fix_df

,ligand,radical,nucleophile,dG
10,L6,R94,N10,-137.650484
30,L7,R94,N10,-159.763919
50,L14,R94,N10,-136.751263
70,L10,R94,N10,-120.560263
80,L4,R20,N10,-135.095893
85,L4,R48,N10,-137.295313
90,L4,R94,N10,-133.909900
91,L4,R94,N61,-157.115829
92,L4,R94,N1,-155.961211
93,L4,R94,N7,-145.202561


In [32]:
other_df = df.drop(fix_df.index)
other_df

,ligand,radical,nucleophile,dG
0,L6,R20,N10,-137.501137
5,L6,R48,N10,-140.315517
7,L6,R48,N1,-161.450664
8,L6,R48,N7,-149.673566
9,L6,R48,N2,-158.911761
11,L6,R94,N61,-159.936484
12,L6,R94,N1,-157.392560
13,L6,R94,N7,-147.113327
14,L6,R94,N2,-151.706069
20,L7,R20,N10,-160.234551


In [33]:
# Populate the coefficient matrix and the right-hand side vector

def get_x_y(df):
    # Create the coefficient matrix and the right-hand side vector
    coefficient_matrix = []
    G = []
    for _, row in df.iterrows():
        if pd.notna(row["dG"]):
            i = np.where(ligand_labels == row["ligand"])[0][0]              # index of fixed ligand
            j = np.where(radical_labels == row["radical"])[0][0]            # index of fixed radical
            k = np.where(nucleophile_labels == row["nucleophile"])[0][0]    # index of fixed nucleophile
            coefficients = [0] * (
                len(ligand_labels) + len(radical_labels) + len(nucleophile_labels)
            )
            coefficients[i] = 1
            coefficients[len(ligand_labels) + j] = 1
            coefficients[len(ligand_labels) + len(radical_labels) + k] = 1
            coefficient_matrix.append(coefficients)
            G.append(row["dG"])

    coefficient_matrix = np.array(coefficient_matrix)
    G = np.array(G)
    G = -G
    return coefficient_matrix, G

In [34]:
# For the ternary linear equation G = L + R + N, there exists a scenario where constant values are assigned to the independent variables (L, R, N), allowing potential linear shifts in their absolute magnitudes.
# For instance, under transformations L' = L + 10, R' = R - 3, N' = N - 3, the equation G = L' + R' + N' still holds true. Different methodologies or initial guesses may lead to variations in these constant value assignments.
# Nevertheless, the relative values of LFER parameters within L, R, and N remain fundamentally consistent.
# Therefore, we adopt the strategy of fixing a single parameter value to obtain precise and stable absolute numerical solutions for L, R, and N.

def solve(matrix, values, fixed_x_val, fixed_y_val, fixed_z_val, mechanism):
    fixed_l_index = np.where(ligand_labels == "L4")[0][0]
    match mechanism:
        case "RE":
            fixed_r_index = len(ligand_labels) + np.where(radical_labels == "R11")[0][0]
            fixed_n_index = len(ligand_labels) + len(radical_labels) + np.where(nucleophile_labels == "N61")[0][0]
        case "RS":
            fixed_r_index = len(ligand_labels) + np.where(radical_labels == "R16")[0][0]
            fixed_n_index = len(ligand_labels) + len(radical_labels) + np.where(nucleophile_labels == "N7")[0][0]
        case "IP":
            fixed_r_index = len(ligand_labels) + np.where(radical_labels == "R94")[0][0]
            fixed_n_index = len(ligand_labels) + len(radical_labels) + np.where(nucleophile_labels == "N10")[0][0]
           
    fixed_indices = [fixed_l_index, fixed_r_index, fixed_n_index]
    fixed_values = [fixed_x_val, fixed_y_val, fixed_z_val]
    A_fixed = matrix[:, fixed_indices]
    adjusted_G = values - np.dot(A_fixed, fixed_values)

    free_columns_mask = np.ones(matrix.shape[1], dtype=bool)
    free_columns_mask[fixed_indices] = False
    A_free = matrix[:, free_columns_mask]

    try:
        params_free, _, _, _ = np.linalg.lstsq(A_free, adjusted_G, rcond=None)
    except np.linalg.LinAlgError:
        print("The matrix cannot be solved.")
        exit()

    solution = np.zeros(matrix.shape[1])
    solution[fixed_indices] = fixed_values
    solution[free_columns_mask] = params_free

    results = pd.DataFrame({
        "Conponent": [ligand_labels[i] for i in range(len(ligand_labels))]
                + [radical_labels[i] for i in range(len(radical_labels))]
                + [nucleophile_labels[i] for i in range(len(nucleophile_labels))],
        "Value": np.concatenate((
            solution[:len(ligand_labels)],
            solution[len(ligand_labels):len(ligand_labels)+len(radical_labels)],
            solution[-len(nucleophile_labels):]
        ))
    })

    return results

In [35]:
from sklearn.model_selection import KFold

mechanism = "IP"
fixed_l_val = 54.6043832717227          # L_IP of L4
fixed_r_val = 73.3557947047808          # R_IP of R94
fixed_n_val = 5.06948540812359          # N_IP of N10

rss = [rs_root * i for i in range(1, 11)]  # random states
fit_results: list[pd.DataFrame] = []
training_r2s = []
training_maes = []
training_rmses = []
test_r2s = []
test_maes = []
test_rmses = []
idxs_1 = []
idxs_2 = []

for rs in rss:
    kf = KFold(n_splits=5, shuffle=True, random_state=rs)
    for fold, (train_idx, test_idx) in enumerate(kf.split(other_df)):

        train_df, test_df = other_df.iloc[train_idx, :], other_df.iloc[test_idx, :]
        combined_df = pd.concat([train_df, fix_df], axis=0)

        x_matrix, values = get_x_y(train_df)
        fit_result = solve(
            x_matrix, values, fixed_l_val, fixed_r_val, fixed_n_val, mechanism
        )

        mapping = fit_result.set_index("Conponent")["Value"].to_dict()

        # combine the fit result L, R, N values
        combined_df[["L", "R", "N"]] = combined_df[
            ["ligand", "radical", "nucleophile"]
        ]
        combined_df.loc[:, ["L", "R", "N"]] = combined_df.loc[:, ["L", "R", "N"]].replace(mapping)
        combined_df[["L", "R", "N"]] = combined_df[["L", "R", "N"]].astype(float)
        combined_df.loc[:, "dG_pred"] = combined_df.loc[:, ["L", "R", "N"]].sum(axis=1).mul(-1)

        test_df[["L", "R", "N"]] = test_df[
            ["ligand", "radical", "nucleophile"]
        ]
        test_df.loc[:, ["L", "R", "N"]] = test_df.loc[:, ["L", "R", "N"]].replace(mapping)
        test_df[["L", "R", "N"]] = test_df[["L", "R", "N"]].astype(float)
        test_df.loc[:, "dG_pred"] = test_df.loc[:, ["L", "R", "N"]].sum(axis=1).mul(-1)
        
        combined_notna_df = combined_df.dropna(axis=0, how="any")
        training_r2 = r2_score(combined_notna_df["dG"], combined_notna_df["dG_pred"])
        training_mae = mean_absolute_error(combined_notna_df["dG"], combined_notna_df["dG_pred"])
        training_rmse = root_mean_squared_error(combined_notna_df["dG"], combined_notna_df["dG_pred"])

        test_notna_df = test_df.dropna(axis=0, how="any")
        try:
            test_r2 = r2_score(test_notna_df["dG"], test_notna_df["dG_pred"])
            test_mae = mean_absolute_error(test_notna_df["dG"], test_notna_df["dG_pred"])
            test_rmse = root_mean_squared_error(test_notna_df["dG"], test_notna_df["dG_pred"])
        except:
            continue

        training_r2s.append(training_r2)
        training_maes.append(training_mae)
        training_rmses.append(training_rmse)
        test_r2s.append(test_r2)
        test_maes.append(test_mae)
        test_rmses.append(test_rmse)

        idxs_1.append(rs//rs_root)
        idxs_2.append(fold)

        fit_results.append(fit_result)

        # fit_result.to_csv(fr"RE_fit_results\solution_output_{mechanism}_round-{rs//1000}_fold-{fold}.csv", index=False)

performance_df = pd.DataFrame({"Round": idxs_1, "Fold": idxs_2, "Training R2": training_r2s, "Traing MAE": training_maes, "Training RMSE": training_rmses, "Test R2": test_r2s, "Test_MAE": test_maes, "Test_RMSE": test_rmses})
performance_df.dropna(axis=0, how="any").describe().round(3).loc["mean", "Test R2"]

/tmp/ipykernel_2119770/2983931022.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df.loc[:, ["L", "R", "N"]] = combined_df.loc[:, ["L", "R", "N"]].replace(mapping)
/tmp/ipykernel_2119770/2983931022.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[["L", "R", "N"]] = test_df[
/tmp/ipykernel_2119770/2983931022.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

0.959

In [36]:
performance_df = pd.DataFrame({"Round": idxs_1, "Fold": idxs_2, "Training R2": training_r2s, "Traing MAE": training_maes, "Training RMSE": training_rmses, "Test R2": test_r2s, "Test_MAE": test_maes, "Test_RMSE": test_rmses})
performance_df.dropna(axis=0, how="any", inplace=True)

In [37]:
performance_df.loc[:, ["Test R2", "Test_MAE","Test_RMSE"]].mean().round(3)

Test R2      0.959
Test_MAE     2.125
Test_RMSE    2.672
dtype: float64

In [38]:
r2s = []
maes = []
rmses = []
for l, d in performance_df.groupby("Round"):
    r2 = d.loc[:, "Test R2"].mean()
    mae = d.loc[:, "Test_MAE"].mean()
    rmse = d.loc[:, "Test_RMSE"].mean()
    r2s.append(r2)
    maes.append(mae)
    rmses.append(rmse)


print(np.array(r2s).mean())
print(np.array(maes).mean())
print(np.array(rmses).mean())
print(np.array(r2s).std())
print(np.array(maes).std())
print(np.array(rmses).std())

0.9591371079379956
2.1250423252896065
2.6724449210298236
0.014376812126853412
0.18449906755551213
0.19104955518324854
